In [1]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.0MB/s]


In [2]:
TEXT.build_vocab(train_data, vectors = "glove.6B.300d", unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                          
100%|█████████▉| 398993/400000 [00:36<00:00, 10914.62it/s]

In [0]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = embedding_dim, out_channels = 50, kernel_size = fs)for fs in [4,5]])
        self.convs2 = nn.Conv1d(in_channels = 50, out_channels = 100, kernel_size = 3)              
        self.fc1 = nn.Linear(149700, 100) 
        self.fc1_bn=nn.BatchNorm1d(100)
        self.fc2=nn.Linear(100,1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        embedded = self.embedding(text)        
        embedded = embedded.permute(0, 2, 1)
        sent_len=embedded.size(2)
        padding=3000-sent_len
        batch_size=embedded.size(0)
        torch_padding=torch.zeros(batch_size,300,padding,dtype = embedded.dtype,device = embedded.device)
        lz=[embedded,torch_padding]
        zcat = torch.cat(lz, dim = 2)
        conved = [F.relu(conv(zcat)) for conv in self.convs]
        pooled = [F.max_pool1d(conv, 2) for conv in conved]
        cat = torch.cat(pooled, dim = 2)
        conved2 = F.relu(self.convs2(cat)) 
        pooled2 = F.max_pool1d(conved2, 2)
        pooled2=pooled2.reshape(batch_size,149700)
        full1= self.fc1(pooled2)
        full1_bn=self.dropout(self.fc1_bn(full1))
        full2= self.fc2(full1)
        return full2

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, DROPOUT, PAD_IDX)

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 45,632,701 trainable parameters


In [36]:
print(model)

CNN1d(
  (embedding): Embedding(101707, 300, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(300, 50, kernel_size=(4,), stride=(1,))
    (1): Conv1d(300, 50, kernel_size=(5,), stride=(1,))
  )
  (convs2): Conv1d(50, 100, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=149700, out_features=100, bias=True)
  (fc1_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [37]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.1834, -0.2495, -0.3197,  ..., -0.0942,  0.0457,  0.6455],
        [-1.5638, -0.1757, -0.6140,  ...,  1.0431,  0.8728, -1.7671],
        [ 1.3687, -0.5770, -0.0523,  ..., -1.5940,  0.3549,  2.0182]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    #Returns accuracy per batch, i.e. for 8/10 right, this returns 0.8 #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train() #to ensure the dropout is "turned on" while training.
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval() #to ensure the dropout is "turned off" while evaluating.
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [43]:
N_EPOCHS = 5
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 47s
	Train Loss: 0.601 | Train Acc: 67.61%
	 Val. Loss: 0.304 |  Val. Acc: 87.55%
Epoch: 02 | Epoch Time: 0m 47s
	Train Loss: 0.237 | Train Acc: 90.75%
	 Val. Loss: 0.263 |  Val. Acc: 89.57%
Epoch: 03 | Epoch Time: 0m 48s
	Train Loss: 0.094 | Train Acc: 96.74%
	 Val. Loss: 0.324 |  Val. Acc: 89.38%
Epoch: 04 | Epoch Time: 0m 48s
	Train Loss: 0.023 | Train Acc: 99.32%
	 Val. Loss: 0.578 |  Val. Acc: 86.52%
Epoch: 05 | Epoch Time: 0m 48s
	Train Loss: 0.015 | Train Acc: 99.57%
	 Val. Loss: 0.579 |  Val. Acc: 88.40%


In [44]:
model.load_state_dict(torch.load('model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.279 | Test Acc: 88.74%


In [0]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [46]:
predict_sentiment(model, "This is undoubtedly the best movie ever!!!")

0.8848834037780762

In [47]:
predict_sentiment(model, "I have to give this zero stars")

0.37059247493743896

In [53]:
predict_sentiment(model, "Good at being terrible ")

0.3468925356864929